# Dummy Tests

In [1]:
from dateparser import parse

parse("today 3 PM")

datetime.datetime(2018, 2, 17, 15, 0)

In [5]:
doc = nlp(sample2)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

today 29 34 DATE
3 PM 38 42 TIME


In [6]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

I -PRON- PRON PRP nsubj X True False
will will VERB MD aux xxxx True True
be be VERB VB aux xx True True
playing play VERB VBG ROOT xxxx True False
Basketball basketball PROPN NNP dobj Xxxxx True False
today today NOUN NN npadvmod xxxx True False
at at ADP IN prep xx True True
3 3 NUM CD nummod d False False
PM pm NOUN NN pobj XX True False
! ! PUNCT . punct ! False False


# Main Code

In [3]:
sample1 = "Did anyone play basketball 1 week and one day ago?"
sample2 = "I will be playing Basketball today at 3 PM!"

In [4]:
import spacy
from dateparser import parse
nlp = spacy.load('en')

In [60]:
import string

def deduce_action(sentence):
    l_sentences = []
    i_pos = 0
    e_pos = 0
    
    while e_pos < len(sentence):
        if sentence[e_pos] in ('.', '!', '?'):
            l_sentences.append(sentence[i_pos:e_pos+1].strip())
            i_pos = e_pos + 1
        e_pos += 1
    if e_pos != i_pos:
        l_sentences.append(sentence[i_pos: e_pos+1].strip())
    
    result = []
    for x in l_sentences:
        result.append(action_per_sentence(x))
    
    return result

def action_per_sentence(sentence):
    
    doc = nlp(sentence)
    
    result_dict = {}
    compound_flag = False
    q = []
    for token in doc:
        if token.dep_ == 'ROOT':
            result_dict['action'] = ' '.join(q + [token.lemma_])
        elif token.dep_ == 'dobj':
            result_dict['what'] = ' '.join(q + [token.text])
        elif token.dep_ == 'nsubj':
            if 'who' not in result_dict:
                result_dict['who'] = ' '.join(q + [token.text])
        elif token.dep_ == 'punct':
            if token.lemma_ == '?':
                result_dict['question'] = True
            else:
                result_dict['question'] = False
        elif token.dep_ == 'pobj':
            if 'who' not in result_dict:
                result_dict['who'] = ' '.join(q + [token.text])
            else:
                result_dict['who'] += " " + ' '.join(q + [token.text])
        elif token.dep_ == 'xcomp':
            result_dict['action'] = ' '.join(q + [token.text])
        
        if token.dep_ == 'compound' or token.dep_ == 'nummod' or \
            token.dep_ == 'npadvmod' or token.dep_ == 'poss' \
            or token.dep_ == 'attr' or token.dep_ == 'prep':
            compound_flag = True
            q.append(token.text)
        else:
            compound_flag = False
            q = []
            
#         print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#           token.shape_, token.is_alpha, token.is_stop)
        
    
    time_measures = []
    locations = []
    people = []
#     print("\n")
    for ent in doc.ents:
        if ent.label_ == 'DATE' or ent.label_ == 'TIME':
            time_measures.append(ent.text)
        elif ent.label_ == 'GPE':
            locations.append(ent.text)
        elif ent.label_ == 'PERSON':
            people.append(ent.text)
        
#         print(ent.text, ent.label_)
    
    date_parse_res = parse(' '.join(time_measures))
#     print(time_measures)
#     print(date_parse_res)
    result_dict['when'] = 'Anytime'
    if date_parse_res is not None:
        result_dict['when'] = date_parse_res
    result_dict['where'] = locations
    result_dict['people'] = people
    
    return result_dict

In [61]:
res = deduce_action("When am I going to get a coop?")

## Type detection

In [67]:
with open('concern_verb_list.txt', 'r') as f:
    concern_verbs = f.read().split()

res = deduce_action("We need to fix the pothole on Road 2.")

if res[0]['action'] in concern_verbs:
    print("Concern!")
else:
    print("Event!")

Concern!
